In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

df = spark.read.option("multiline", "true").json("Files/news/latest_news.json")
# df now is a Spark DataFrame containing JSON data from "Files/news/latest_news.json".
display(df)

SynapseWidget(Synapse.DataFrame, 07076b68-a7a5-4fd0-900d-ca339ffde9c3)

In [2]:
df = spark.read.option("multiline", "true").json("Files/news/latest_news.json")
# df now is a Spark DataFrame containing JSON data from "Files/news/latest_news.json".
# df.show()

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 4, Finished, Available, Finished)

In [3]:
df=df.select("results")

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 5, Finished, Available, Finished)

In [4]:
from pyspark.sql.functions import explode
df_exploded=df.select(explode(df["results"]).alias("json_object"))

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 6, Finished, Available, Finished)

In [5]:
df_exploded.show()

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 7, Finished, Available, Finished)

+--------------------+
|         json_object|
+--------------------+
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
|{ONLY AVAILABLE I...|
+--------------------+



In [6]:
json_list=df_exploded.toJSON().collect()
# print(json_list[0])

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 8, Finished, Available, Finished)

In [7]:
import json
news_json=json.loads(json_list[4])
print(news_json)

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 9, Finished, Available, Finished)

{'json_object': {'ai_org': 'ONLY AVAILABLE IN CORPORATE PLANS', 'ai_region': 'ONLY AVAILABLE IN CORPORATE PLANS', 'ai_summary': 'ONLY AVAILABLE IN PAID PLANS', 'ai_tag': 'ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE PLANS', 'article_id': '264624e8557c815a2a461de6c064dd1e', 'category': ['lifestyle', 'top'], 'content': 'ONLY AVAILABLE IN PAID PLANS', 'country': ['india'], 'creator': ['tribune news service'], 'datatype': 'news', 'description': "Dr. Dharminder Kumar, head of KVK Chamba, received the 'Best KVK Head Award' at the Global Agricultural Conference in Chandigarh. The award recognized his leadership and contributions to agricultural advancements, farmer empowerment, and implementing innovative farming practices in Chamba district.", 'duplicate': False, 'fetched_at': '2026-02-22 16:38:15', 'image_url': 'https://www.tribuneindia.com/sortd-service/imaginary/v22-01/webp/medium/high?url=dGhldHJpYnVuZS1zb3J0ZC1wcm8tcHJvZC1zb3J0ZC9tZWRpYWJjMDEzODcwLTEwMGEtMTFmMS1hN2NkLWExNzYyOTNmNDNjZS5q

In [13]:
title=[]
description=[]
category=[]
url=[]
image=[]
source=[]
datepublished=[]

for json_str in json_list:
    
    try:
        article=json.loads(json_str)
        if article["json_object"].get("image_url"):
            title.append(article["json_object"]["title"])
            description.append(article["json_object"]["description"])
            category.append(article["json_object"]["category"])
            url.append(article["json_object"]["link"])
            image.append(article["json_object"]["image_url"])
            source.append(article["json_object"]["source_name"])
            datepublished.append(article["json_object"]["pubDate"])
    except Exception as e:
        print(f"Error processing JSON object: {e}")
print(title)

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 15, Finished, Available, Finished)

['Evaluating Indian Kings: Tipu Sultan', 'Will not abandon Kaleshwaram irrigation project: Telangana CM Revanth', 'Kondapur to get govt college after HYDRAA clears encroachments', 'Amazon’s Global Legal Head Visits Bhubaneswar’s KIIT & KISS; Conferred Lifetime Achievement Award', 'Chamba scientist honoured at global conference', "Jilted lover held for issuing threat to blow up court complex in UP's Jaunpur ATS", "Jilted lover held for issuing threat to blow up court complex in UP's Jaunpur: ATS", 'Astronauts Will Be Allowed to Use the Latest Smartphones During Space Missions', 'Changes in GDP Data Can Impact Many Things', 'India restrict South Africa to 187/7 despite fine knocks from Miller and Brevis']


In [9]:
from pyspark.sql.types import StructType, StructField, StringType

data=list(zip(title,description,category,url,image,source,datepublished))

schema=StructType([
    StructField("title",StringType(), True),
    StructField("description",StringType(), True),
    StructField("category",StringType(), True),
    StructField("url",StringType(), True),
    StructField("image",StringType(), True),
    StructField("source",StringType(), True),
    StructField("datepublished",StringType(), True),
])

df_cleaned = spark.createDataFrame(data,schema=schema)
df_cleaned.show()

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 11, Finished, Available, Finished)

+--------------------+--------------------+------------------+--------------------+--------------------+-----------------+-------------------+
|               title|         description|          category|                 url|               image|           source|      datepublished|
+--------------------+--------------------+------------------+--------------------+--------------------+-----------------+-------------------+
|Evaluating Indian...|An analytical eva...|        [domestic]|https://ismatimes...|https://ismatimes...|       Isma Times|2026-02-22 16:29:37|
|Will not abandon ...|Hyderabad: In a m...|      [crime, top]|https://www.siasa...|https://cdn.siasa...|       Siasat.com|2026-02-22 16:25:14|
|Kondapur to get g...|Hyderabad: After ...|[top, environment]|https://www.siasa...|https://cdn.siasa...|       Siasat.com|2026-02-22 16:23:49|
|Amazon’s Global L...|Bhubaneswar: KIIT...|  [lifestyle, top]|https://odishabyt...|https://assets.od...|Odisha Bytes News|2026-02-22 16:23:16|

In [10]:
from pyspark.sql.functions import to_date,date_format
df_final_cleaned=df_cleaned.withColumn("datepublished",date_format(to_date("datepublished"),"dd-MMM-yyyy"))
df_final_cleaned.show()

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 12, Finished, Available, Finished)

+--------------------+--------------------+------------------+--------------------+--------------------+-----------------+-------------+
|               title|         description|          category|                 url|               image|           source|datepublished|
+--------------------+--------------------+------------------+--------------------+--------------------+-----------------+-------------+
|Evaluating Indian...|An analytical eva...|        [domestic]|https://ismatimes...|https://ismatimes...|       Isma Times|  22-Feb-2026|
|Will not abandon ...|Hyderabad: In a m...|      [crime, top]|https://www.siasa...|https://cdn.siasa...|       Siasat.com|  22-Feb-2026|
|Kondapur to get g...|Hyderabad: After ...|[top, environment]|https://www.siasa...|https://cdn.siasa...|       Siasat.com|  22-Feb-2026|
|Amazon’s Global L...|Bhubaneswar: KIIT...|  [lifestyle, top]|https://odishabyt...|https://assets.od...|Odisha Bytes News|  22-Feb-2026|
|Chamba scientist ...|Dr. Dharminder Ku..

In [11]:
# df_final_cleaned.write.format("delta").saveAsTable("silver.news_data")

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 13, Finished, Available, Finished)

In [12]:
from pyspark.sql.utils import AnalysisException
try:
    table_name= "silver.news_data"
    df_final_cleaned.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table already exists")

    df_final_cleaned.createOrReplaceTempView("vw_df_final_cleaned")
    
    spark.sql(f""" MERGE INTO {table_name} target_table
                    USING vw_df_final_cleaned view_table
                    ON target_table.url=view_table.url
                    WHEN MATCHED AND
                    target_table.title<>view_table.title OR
                    target_table.description<>view_table.description OR
                    target_table.category<>view_table.category OR
                    target_table.image<>view_table.image OR
                    target_table.source<>view_table.source OR
                    target_table.datepublished<>view_table.datepublished
                    THEN UPDATE SET *
                    WHEN NOT MATCHED THEN INSERT * """)

StatementMeta(, 37d575e3-6df3-481e-9123-0ef8ad815a76, 14, Finished, Available, Finished)

Table already exists
